In [1]:
from lxml import html
import requests
from string import ascii_lowercase
import sys
import json
from xml.etree.ElementTree import tostring
from IPython.display import clear_output
from time import sleep

# base URL
page_root_url = 'http://www.laughfactory.com/jokes/'
# file name for the topics
topics_filename = 'topicsLF.json'
# filename for the jokes
jokes_filename = 'jokesLF.json'
# max joke length
MAX_JOKE_LENGTH = 1000

In [2]:
# request HTML page
page_response = requests.get(page_root_url)
# convert to tree for searching for elements with xpath
tree = html.fromstring(page_response.content)

topics_names = tree.xpath('//div[@class="jokes-nav"]//ul//li//text()')
topics_urls = tree.xpath('//div[@class="jokes-nav"]//ul//li//a/@href')
topics = {}
ix = 0
for topic_name in topics_names:
    topics[topic_name] = topics_urls[ix]
    ix += 1

with open(topics_filename, 'w') as fp:
    json.dump(topics, fp)

In [3]:
with open(topics_filename, 'r') as fp:
    topics = json.load(fp)  
topic_names = sorted(topics.keys())
joke_topic_names = [topic_names[0]]

jokes = {}
tot_n_jokes = 0
try:    
    with open(jokes_filename, 'r') as fp:
        jokes = json.load(fp)    
    if len(jokes) == 0:
        print('File does not contain jokes of jokes dict is empty')
    else:
        topics_without_jokes = []
        for topic in topics:
            if topic in jokes.keys():
                n_jokes_topic = 0
                jokes_topic = jokes[topic]
                for joke in jokes_topic:
                    n_jokes_topic+=1
            else:
                topics_without_jokes.append(topic)
            tot_n_jokes += n_jokes_topic
except:
    print('Could not find jokes file: ' + jokes_filename)
    
print('Total number of jokes: ' + str(tot_n_jokes))
curr_topic = joke_topic_names[-1]
print('Current topic: ' + curr_topic)
print('Last topic: ' + topic_names[-1])

Total number of jokes: 102620
Current topic: Animal Jokes
Last topic: Yo Momma Jokes


In [4]:
redirect_joke = ''
final_page_url = 'http://www.laughfactory.com/jokes'
page_response = requests.get(final_page_url, stream=True)
tree = html.fromstring(page_response.content)
jokes_tree = tree.xpath('//div[@class="joke-text"]')         
joke = jokes_tree[0]
joke_text_list = [x for x in joke.itertext()] 
formatted_joke = ' '.join(joke_text_list).strip()
redirect_joke = formatted_joke
print('Redirect joke: ' + redirect_joke)

Redirect joke: I asked a Chinese girl for her number. She said, "Sex! Sex! Sex! Free sex tonight!" I said, "Wow!" Then her friend said, "She means 666-3629."


In [18]:
def get_topic_jokes(topic,topic_page_url,jokes, tot_n_jokes = 0):
    MAX_RANGE_PAGE = 1000
    jokes_list = []   
    ix_joke = 0
    redirected_joke = False
    for ix_page in range(1,MAX_RANGE_PAGE):
        final_page_url = topic_page_url + '/' + str(ix_page)
        page_response = requests.get(final_page_url, stream=True)
        tree = html.fromstring(page_response.content)
        jokes_tree = tree.xpath('//div[@class="joke-text"]')
        if len(jokes_tree) == 0:
            print('Found last joke page')
            break        
        else:            
            for joke in jokes_tree:
                ix_joke += 1
                joke_text_list = [x for x in joke.itertext()] 
                formatted_joke = ' '.join(joke_text_list).strip()
                if formatted_joke == redirect_joke and tot_n_jokes > 0:
                    redirected_joke = True
                    print('Redirected - stopping topic')
                    break
                jokes_list.append(formatted_joke)                           
                tot_n_jokes += 1
                print('Total number of jokes: ' + str(tot_n_jokes))
                print(final_page_url)
                print(topic + ': ' + formatted_joke)
                clear_output(wait=True)
        if redirected_joke:
            break    
    jokes[topic] = jokes_list
    return jokes, tot_n_jokes 

In [ ]:
tot_n_jokes = 0
for topic in topics:  
    jokes, tot_n_jokes = get_topic_jokes(topic,topics[topic],jokes,tot_n_jokes)
    with open(jokes_filename, 'w') as fp:
        json.dump(jokes, fp)
        
with open(jokes_filename, 'r') as fp:
    jokes = json.load(fp)  

Total number of jokes: 3093
http://www.laughfactory.com/jokes/word-play-jokes/28
Word Play Jokes: Knock knock.  Who's there?  Marry.  Marry who?  Marry me, please!
